# Word Embeddings
* Author: Johannes Maucher
* Last Update: 26.02.2018


## Representations of single Words
There are different ways to represent single words at the input of a machine learning algorithm. The conventional (old) way is to apply a **one-hot-encoding**. In this approach each word is represented by a vector $\mathbf{v_w}$, whose length is given by the number of words in the applied vocabulary $V$. Each word $w \in V$ is uniquely mapped to an integer $i_w \in [0,|V|]$ and the one-hot encoded word-vector $\mathbf{v_w}$ contains a 1 at position $i_w$ and zeros at all other positions. Drawbacks of the one-hot representation of words are:
* the one-hot encoded vectors are very long and sparse
* semantic or syntactic relations between words, can not be infered from the corresponding one-hot encoded vectors. 

The better way of representing words is to apply a **word-embedding**. Word embeddings have revolutionalized many fields of Natural Language Processing since their efficient neural-network-based generation has been published in [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf) by Mikolov et al (2013). Word embeddings map words into vector-spaces such that semantically or syntactically related words are close together, whereas unrelated words are far from each other. Moreover, it has been shown that the word-embeddings, generated by *word2vec*-techniques *CBOW* or *Skipgram*, are well-structured in the sense that also relations such as *is-capital-of*, *is-female-of*, *is-plural-of* are encoded in the vector space. In this way questions like *woman is to queen, as man is to ?* can be answered by simple operations of linear algebra in the word-vector-space. Compared to the length of one-hot encoded word-vectors, word-embedding-vectors are short (typical lengths in the range from 100-300) and dense (float-values). 

![DSM concept](./Pics/dsm.png)

*CBOW* and *Skipgram*, are techniques to learn word-embeddings, i.e. a mapping of words to vectors by relatively simple neural networks. Usually large corpora are applied for learning, e.g. the entire Wikipedia corpus in a given language. Today, pretrained models for the most common languages are available, for example from this [Facebook github repository](https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md?utm_campaign=buffer&utm_content=buffer0df9b&utm_medium=social&utm_source=linkedin.com).

For the classifier implemented in [notebook K04germanNewsFeedClassification.ipynb](K04germanNewsFeedClassification.ipynb), a self-trained german word-embedding is applied. The embedding has been trained with CBOW from the dump of the entire german Wikipedia in notebook [K03generateCBOWfromWiki.ipynb](K03generateCBOWfromWiki.ipynb)


## Representations of Documents

### Bag-of-Word model
The conventional way of modelling documents in tasks like information-retrieval, document-clustering, document-classification, sentiment-analysis, topic-classification is to represent each document as a **Bag-Of-Word**-vector $$\mathbf{d}_i=(tf_{i,0},tf_{i,1},\ldots tf_{i,|V|}).$$ Each component of this vector corresponds to a single term $j$ of the underlying vocabulary $V$ and the values $tf_{i,j}$ counts the frequency of term $j$ in document $i$. Instead of the term-frequency $tf_{i,j}$ it is also possible to fill the BoW-vector with 
* a binary indicator which indicates if the term $j$ appears in document $i$
* the tf-idf-values $$tfidf_{i,j}=tf_{i,j} \cdot log \frac{N}{df_j},$$ where $df_j$ is the frequency of documents, in which term $j$ appears, and $N$ is the total number of documents.

Independent of the values used, the BoW-model has the following major drawbacks:
* the order by which terms appear in the document is totally ignored
* semantic relatedness of terms is not modelled
* BoW- vectors are very long and sparse

### Sequences of Word-Vectors
Using word-embeddings all of the mentioned drawbacks of BoW-modells can be avoided. One approach is to represent the sequence of words in the document as a sequence of word-embedding vectors. These sequences can then be passed to e.g. Convolutional Neural Networks (CNN) or Long-Short-Term-Memory Networks (LSTM).  

![BoWvsEmbedding](./Pics/bowVsEmbedding.png)

### Vector-Representations of Sentences, Paragraphs and Documents
Word-Embeddings are based on the idea, that semantically related words can be mapped to vectors, which are close together. Where two words are semantically related, if they frequently appear in the same context (neighbouring words). This idea can also be applied to sentences or word-sequences in general. By applying e.g. LSTMs, Doc2Vec, Encoder-Decoder architectures etc. vector representations of texts can be generated, such that semantically related texts have similiar vector-representations. 


## CBOW- and Skipgram- Wordembedding
In 2013 Mikolov et al. published [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf). They proposed quite simple neural network architectures to efficiently create word-embeddings: CBOW and Skipgram. These architectures are better known as **Word2Vec**. In both techniques neural networks are trained for a pseudo-task. After training, the network itself is usually not of interest. However, the learned weights in the input-layer constitute the word-embeddings, which can then be applied for a large field of NLP-tasks, e.g. document classification.

### Continous Bag-Of-Words (CBOW)
The idea of CBOW is to predict the target word $w_i$, given the $N$ context-words $w_{i-N/2},\ldots, w_{i-1}, \quad w_{i+1}, w_{i+N/2}$. 
In order to learn such a predictor a large but unlabeled corpus is required. The extraction of training-samples from a corpus is sketched in the picture below:

![cbowTrainSamples](./Pics/cbowTrainSamples.png)

In this example a context length of $N=4$ has been applied. The first training-element consists of 
* the $N=4$ input-words *(happy,families,all,alike)*
* the target word *are*.

In order to obtain the second training-sample the window of length $N+1$ is just shifted by one to the right. The concrete architecture for CBOW is shown in the picture below. At the input the $N$ context words are one-hot-encoded. The fully-connected *Projection-layer* maps the context words to a vector representation of the context. This vector representation is the input of a softmax-output-layer. The output-layer has as much neurons as there are words in the vocabulary $V$. Each neurons uniquely corresponds to a word of the vocabulary and outputs an estimation of the probaility, that the word appears as target for the current context-words at the input.  
![cbowArchitecture](./Pics/cbowGramArchitecture.png)

After training the CBOW-network the vector representation of word $w$ are the weights from the one-hot encoded word $w$ at the input of the network to the neurons in the projection-layer. I.e. the number of neurons in the projection layer define the length of the word-embedding.

### Skip-Gram
Skip-Gram is similar to CBOW, but has a reversed prediction process: For a given target word at the input, the Skip-Gram model predicts words, which are likely in the context of this target word. Again, the context is defined by the $N$ neighbouring words. The extraction of training-samples from a corpus is sketched in the picture below:

![skipGramTrainSamples](./Pics/skipGramTrainSamples.png)

Again a context length of $N=4$ has been applied. The first training-element consists of 
* the first target word *(happy)* as input to the network 
* the first context word *(families)* as network-output.

The concrete architecture for Skip-gram is shown in the picture below. At the input the target-word is one-hot-encoded. The fully-connected *Projection-layer* outputs the current vector representation of the target-word. This vector representation is the input of a softmax-output-layer. The output-layer has as much neurons as there are words in the vocabulary $V$. Each neurons uniquely corresponds to a word of the vocabulary and outputs an estimation of the probaility, that the word appears in the context of the current target-word at the input.  
![cbowArchitecture](./Pics/skipGramArchitecture.png)

## Other Word-Embeddings
CBOW- and Skip-Gram are possibly the most popular word-embeddings. However, there are more count-based and prediction-based methods to generate them, e.g. Random-Indexing, [Glove](https://nlp.stanford.edu/projects/glove/), [FastText](https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md).

### Integration of Fasttext Word-Embeddings
After downloading word embeddings from [FastText](https://fasttext.cc/docs/en/english-vectors.html) they can be imported as follows:

In [1]:
!pip install --upgrade gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 33.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.9 MB/s eta 0:00:00


In [2]:
from __future__ import print_function
from gensim.models import KeyedVectors

# Creating the model
en_model = KeyedVectors.load_word2vec_format('/Users/maucher/DataSets/Gensim/FastText/wiki-news-300d-1M.vec')

# Getting the tokens 
words = []
for word in en_model.vocab:
    words.append(word)

# Printing out number of tokens available
print("Number of Tokens: {}".format(len(words)))

# Printing out the dimension of a word vector 
print("Dimension of a word vector: {}".format(
    len(en_model[words[0]])
))

Number of Tokens: 1000000
Dimension of a word vector: 300


In [3]:
print(words[100])
print("First 10 components of word-vector: \n",en_model[words[100]][:10])

than
First 10 components of word-vector: 
 [ 0.1016 -0.1216 -0.0356  0.0096 -0.1015  0.1766 -0.0593  0.032   0.0892
 -0.0727]


### Glove Word-Embeddings
After downloading word-embeddings from [Glove](https://nlp.stanford.edu/projects/glove/), they can be imported as follows:

In [5]:
import os
import numpy as np
#GLOVE_DIR = "./Data/glove.6B"
GLOVE_DIR = "/Users/maucher/DataSets/glove.6B"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


## Examples of Semantic Relatedness from Word-Embeddings
### Visualisation through t-SNE projection in 2-dim space

![SemanticRelatednessVis](./Pics/semanticRelatednessVis.png)

Source: [http://metaoptimize.s3.amazonaws.com/cw-embeddings-ACL2010/embeddings-mostcommon.EMBEDDING_SIZE=50.png](http://metaoptimize.s3.amazonaws.com/cw-embeddings-ACL2010/embeddings-mostcommon.EMBEDDING_SIZE=50.png)

### Find most similar words
![collobertSimilarities](./Pics/collobertSimilarities.png).

Source: [R. Collobert, NLP (almost) from Scratch](https://arxiv.org/pdf/1103.0398v1.pdf)

### Vector representations of semantic relations
![semanticRelations](./Pics/semanticRelations.png)

Source: [T. Mikolov et al, Linguistic Regularities in Continuous Space Word Representations](https://www.aclweb.org/anthology/N/N13/N13-1090.pdf)

### Lexical Contrast Injection
![lexicalContrast](./Pics/lexicalContrastInjection.png)
Source: [Pham et al, A Multitask Objective to Inject Lexical Contrast into Distributional Semantics](https://www.aclweb.org/anthology/P15-2004)


### Multilingual Embeddings
![Bilingual Embedding](./Pics/bilingualEmbedding.png)
Source: [Ruder et al, A Survey of Cross-lingual Word Embedding Models](https://arxiv.org/pdf/1706.04902.pdf)